# Домашнее задание 5. Линейные модели

In [21]:
import random as pr
import numpy as np
import pandas as pd
import matplotlib.pylab as pl
import sklearn.cross_validation as cv
import sklearn.metrics as sm

# Plotting config
%pylab inline

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


Зачитываем результат 4 домашки

In [22]:
data = np.load("files/out_4.dat.npz")
users = data["users"]
X = data["data"].reshape(1,)[0]

Зачитываем категории пользователей

In [23]:
TRAINING_SET_URL = "twitter_train.csv"
df_users = pd.read_csv(TRAINING_SET_URL, sep=",", header=0, names=["twitter_id", "is_1", "is_2", "is_3"], dtype={"twitter_id": str, "is_1": int, 'is_2': int, "is_3": int})
df_users.set_index("twitter_id", inplace=True)

Формируем целевую переменную: Делаем join списка пользователей из ДЗ4 с обучающей выборкой.

In [24]:
def f(x):
    if x[0] == 1:
        return 1
    if x[1] == 1:
        return 2
    if x[2] == 1:
        return 3

Y = df_users[['is_1', 'is_2', 'is_3']].apply(f, axis=1).values
print "Resulting training set: (%dx%d) feature matrix, %d target vector" % (X.shape[0], X.shape[1], Y.shape[0])

Resulting training set: (3001x70832) feature matrix, 3000 target vector


Чтобы исследовать, как ведут себя признаки, построим распределение количества ненулевых признаков у пользователей, чтобы убедиться, что он удовлетворяет закону Ципфа. Для этого построим гистограмму в логарифмических осях. [Подсказка](http://anokhin.github.io/img/sf1.png)

In [25]:
def draw_log_hist(x):
    """Draw tokens histogram in log scales"""
    
    # Your code here
    
    return [0] * x.shape[1]

features_counts = draw_log_hist(X)

Проведем отбор признаков. В самом простом случае просто удаляем признаки, имеющие ненулевое значение у менее, чем 100 пользователей.

In [26]:
X1 = X.tocsc()[:, features_counts > 100].toarray()

Вариант задания генерируется на основании вашего ника в техносфере.

In [27]:
USER_NAME = "n.anokhin"
OPTIMIZATION_ALGORITHMS = ["stochastic gradient descent", "Newton method"]
REGULARIZATIONS = ["L1", "L2"]

print "My homework 5 algorithm is: Logistic regression with %s regularization optimized by %s" % (
    REGULARIZATIONS[hash(USER_NAME) % 2],
    OPTIMIZATION_ALGORITHMS[hash(USER_NAME[::-1]) % 2]
)

My homework 5 algorithm is: Logistic regression with L2 regularization optimized by Newton method


Реализуем выбранный алгоритм

In [28]:
class LogisticRegression():
    
    def fit(self, X, Y=None):
        return self
    
    def predict_proba(self, X):
        import numpy.random as nr
        return nr.random((X.shape[0], 2))

Реализуем метрику качества, используемую в соревновании: площадь под ROC кривой

In [29]:
def auroc(y_prob, y_true):
    return 0.5

Разделим выборку с помощью методики кросс-валидации для того, чтобы настроить параметр регуляризации $C$

In [30]:
C = [0.0, 0.01, 0.1, 1, 10, 100, 1000, 10000]

def select_reg_parameter(C, X, Y):
    return C.index(max(C))

index = select_reg_parameter(C, X1, Y)
print index


7


Выбираем наилучшее значение $C$, и классифицируем неизвестных пользователей и строим ROC-кривую

In [36]:
def predict(X, Y, test_size, C):
    Y_test = [1,2,3]*300
    Y_prob = [[0.1,0.2,0.7]]*900
    return Y_test, Y_prob

def roc(Y_test, Y_prob, y_prob_ind, pos_label):
    tpr = [1] * 2400
    fpr = [0.01] * 2400
    roc_auc = 0.51
    
    return tpr, fpr, roc_auc

def plot_roc_curve(tpr, fpr, roc_auc):
    """Plot ROC curve"""
    # Your code here
    return

Y_test, Y_prob = predict(X1, Y_1, 0.3, C[index1])

tpr, fpr, roc_auc = roc(Y_test, Y_prob, 0, 1)
print "Area under the ROC curve : %f" % roc_auc
plot_roc_curve(tpr, fpr, roc_auc)

tpr, fpr, roc_auc = roc(Y_test, Y_prob, 1, 2)
print "Area under the ROC curve : %f" % roc_auc
plot_roc_curve(tpr, fpr, roc_auc)

tpr, fpr, roc_auc = roc(Y_test, Y_prob, 2, 3)
print "Area under the ROC curve : %f" % roc_auc
plot_roc_curve(tpr, fpr, roc_auc)

Area under the ROC curve : 0.510000
Area under the ROC curve : 0.510000
Area under the ROC curve : 0.510000


С помощью полученной модели предсказываем категории для неизвестных пользователей из соревнования и загружаем на kaggle в нужном формате.